In [5]:
# Importar librerías 

import pandas as pd  

from bs4 import BeautifulSoup
import requests

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from time import sleep  

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import re


In [6]:
class ExtraerDatosActoresSelenium:
    def __init__(self, archivo_csv): 
        self.archivo_csv = pd.read_csv(archivo_csv)  
        self.info_actores = [] 
        self.info_actores_tupla = [] 

    def pag_principal_google(self):
        # Inicializar el driver de Chrome
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        self.driver = webdriver.Chrome(options=options)
        sleep(2)
        self.driver.maximize_window()
        self.driver.get("https://www.google.es/")
        # acepta cookies
        sleep(5)
        try:
            self.driver.find_element("css selector", "#L2AGLb").click()
        except:
            pass
        self.escoger_actor_csv()         

    def escoger_actor_csv(self):  
        
        # Itera por cada pelicula del csv
        for peli in range(0, 10):  
            # Escoge info del csv para la busqueda  
            actor = (f"{self.archivo_csv['actor'][peli]}")  
            # Mete en la lista el actor  
            self.info_actores.append(actor)              
            self.busqueda_google("imdb", actor)
            self.busqueda_google("imdb premios", actor)
            tupla = tuple(self.info_actores)
            self.info_actores_tupla.append(tupla)
            self.info_actores.clear()  
            
            self.guardar_info_actores_csv("info_actores")
       
    def busqueda_google(self, pagina, dato_buscador):
        sleep(3)
        # Ingresar el título y año de la película en el campo de búsqueda
        self.driver.find_element("css selector", "#APjFqb").clear()
        self.driver.find_element("css selector", "#APjFqb").send_keys(f'{pagina} "{dato_buscador}"', Keys.ENTER)             
       
        # Definimos la url de la página de la que vamos a sacar datos y llama a la api   
        try:            
            sleep(5)            
            href = self.driver.find_element("css selector", "#rso .yuRUbf > div > span > a").get_attribute("href")   
            # si el primer resultado de google no es de imdb pasa a la siguiente pelicula 
            if "imdb" in href:                                
                self.llamada_api(href) 
            else:
                self.info_actores.append("")
                print(f"ha fallado la llamada a la pagina {pagina}")
                
        except:
            pass    
    
    def llamada_api (self, href):

        # Definimos un User-Agent para que la solicitud parezca venir de un navegador real
        # Elegimos el headers en funcion de si vamos a llamar a imdb o rotten
        
        headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
              

        # Hacemos la request a la página de la que queremos sacar la info
        resultado = requests.get(href, headers=headers)

        # Vemos si todo ha ido bien
        print("La respuesta de la petición es:", resultado.status_code)

        # creamos el objeto BeautifulSoup para poder acceder al contenido solicitado
        sopa = BeautifulSoup(resultado.content, 'html.parser')
        # si el objeto viene de imdb llamo a su método para limpiar los datos
        if resultado.status_code == 200 and "award" in href:                
            self.limpieza_datos_premios_imdb(sopa)  
        elif resultado.status_code == 200 :                
            self.limpieza_datos_actores_imdb(sopa)          
        else:
            return 
    
    def limpieza_datos_actores_imdb(self, sopa):
                
        # sacamos año nacimiento        
        anho_nacimiento = sopa.find_all("span", {"class": "sc-59a43f1c-2"})
        self.info_actores.append(anho_nacimiento[1].text)
        
        # pq es conocido
        conocido = sopa.find_all("a", {"class": "ipc-metadata-list-summary-item__t"})
        lista =[]
        for i in range(0,10):   
            lista.append(conocido[i].text)
        self.info_actores.append(lista)

        # que hace
        que_hace = sopa.find_all("ul", {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"})
        self.info_actores.append(que_hace[0].text)  

    def limpieza_datos_premios_imdb(self, sopa):
        premios = []
        # premios
        premios = sopa.find_all("a", {"class": "ipc-metadata-list-summary-item__t"})
        for i in range(0,4):
            premios[i].text
            premios.append(premios[i].text)
        self.info_actores.append(premios)
        print(len(self.info_actores))

    def guardar_info_actores_csv (self, nombre_archivo):
        try:
            df_actores = pd.DataFrame(self.info_actores_tupla, columns=["nombre_apellido", "anho_nacimiento", "peliculas", "que_hace", "premios"])
            df_actores.to_csv(nombre_archivo+".csv", index=False, header=True)
        except:
            pass

       

In [7]:
csv = "info_peliculas_Drama_actores.csv"
# csv = input("Introduce el género del que quieres conseguir la información")
try:
    peliculas_accion_sele = ExtraerDatosActoresSelenium(f"{csv}")
    peliculas_accion_sele.pag_principal_google()
except:
    pass
    # print("Hay algún error, comprueba que exista el fichero")


La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200
La respuesta de la petición es: 200


In [8]:
peliculas_accion_sele.info_actores_tupla

[('Cesario Augusto',),
 ('Lloyd DeLeon',),
 ('Arlette Torres',
  'July 5',
  ['Noria',
   'La pasión turca',
   'Laura y el misterio de la novia que esperó demasiado',
   'Tintin Reporter: Cigars of the Pharaoh',
   'Desmadre incluido',
   'Élite',
   'Hotel Colón: Confinamiento incluído',
   'By Ana Milán',
   'Madres. Amor y vida',
   'Señoras del (h)AMPA'],
  'Actress'),
 ('Ramón Rodríguez',
  'December 20, 1979',
  ['Will Trent: Agente Especial',
   'G20',
   'Olga Dies Dreaming',
   'Will Trent: Agente Especial',
   'Canción de Cuna',
   'Our Time',
   'El magnífico Iván',
   'Prism',
   'The Affair',
   'The Defenders'],
  'ActorProducerDirector'),
 ('Jim Ortlieb',
  'June 19, 1956',
  ['Baby Blues - Going Dark',
   'The Walkers',
   'Next Exit',
   'Estación 19',
   'American Horror Story',
   '9-1-1: Lone Star',
   'Chicago Med',
   'The Rookie',
   'The American Ambassador',
   'The Green Ribbon'],
  'Actor'),
 ('Sam Bilinkas',),
 ('Pedro Lander',
  'March 3, 1954',
  ['Interf

In [9]:
archivo_csv = pd.read_csv("info_peliculas_Action_actores.csv") 
info_actores = []
info_actores_tupla = []
for peli in range(0, len(archivo_csv)):  
            # Escoge info del csv para la busqueda   
            actor = (f"{archivo_csv['actor'][peli]}")  
            print(actor)
            # Mete en la lista el id     
            info_actores.append(actor)   
            print(info_actores)           
            print("imdb", actor)
            # Comprueba que la lista peliculas tenga mas de un dato (evitar que solo tenga el dato de tomatoes) lo pasa a la lista de tuplas y resetea el contenido           
            if len(info_actores) > 0:
                tupla = tuple(info_actores)
                info_actores_tupla.append(tupla)
                print(info_actores_tupla)
            info_actores.clear()
            break

Charlee Fraser
['Charlee Fraser']
imdb Charlee Fraser
[('Charlee Fraser',)]


In [10]:
import requests

# Definimos la url de la página de la que vamos a sacar datos
url_imbd = "https://www.imdb.com/name/nm0000093/awards/"

# Definimos un User-Agent para que la solicitud parezca venir de un navegador real
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Hacemos la request a la página de la que queremos sacar la info
res_moviles = requests.get(url_imbd, headers=headers)

# Vemos si todo ha ido bien
print("La respuesta de la petición es:", res_moviles.status_code)


La respuesta de la petición es: 200


In [11]:
# creamos el objeto BeautifulSoup para poder acceder al contenido solicitado
sopa_peliculas = BeautifulSoup(res_moviles.content, 'html.parser')
sopa_peliculas

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1721158066810);
        }
    })</script><title>Brad Pitt - Awards - IMDb</title><meta content="Brad Pitt - Awards - IMDb - Awards, nominations, and wins" data-id="main" name="description"/><meta content="Brad Pitt, Awards, Awards, Nominations, Oscars, MTV Movie Awards, Emmys, Won, Winner, Nominated, Nominee" n

In [12]:
premios = sopa_peliculas.find_all("a", {"class": "ipc-metadata-list-summary-item__t"})
premios[0].text


'2020 Winner Oscar'

In [13]:
pagina_premios = sopa_peliculas.find_all("a", {"class": "ipc-metadata-list-item__icon-link"})
a = pagina_premios[1]

for a in pagina_premios:
    href_value = a.get('href')
    print(href_value)

https://help.imdb.com/article/contribution/other-submission-guides/awards/G5KGRJURZFQGHJQH#/?ref_=nmawd_cn_awg
https://contribute.imdb.com/czone?ref_=nmawd_cn_cz


In [14]:
que_hace = sopa_peliculas.find_all("ul", {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"})
texto = que_hace[0].text
texto_modif = re.sub(r'([a-z])([A-Z])', r'\1, \2', texto)
texto_modif

IndexError: list index out of range

In [ ]:
conocido = sopa_peliculas.find_all("a", {"class": "ipc-metadata-list-summary-item__t"})
lista =[]
for i in range(0,10):   
    lista.append(conocido[i].text)
lista

['Furiosa: De la saga Mad Max',
 'Cualquiera menos tú',
 'The skin I live in',
 'Vogue Italia: Curtain Call',
 'Vogue Arabia: Rush Hour',
 'Vogue Portugal: A Lesson in Luxury',
 'Mango: A Summer Story',
 'Tory Burch Holiday 2017: Coming Home',
 'Rag & Bone X Star Wars',
 'Tom Ford: Boys & Girls']